In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt

import time
import os
import cv2
from PIL import Image

from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

C:\Users\jeffg\AppData\Local\Temp\ipykernel_21804\2011928270.py:19: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
now_path = os.getcwd()
os.chdir(r"..\..\face-api\jupyter")
import __init__
from faceapi import FaceAPIManager
face_api_manager = FaceAPIManager(
    detector_model_path=r'C:\Users\jeffg\face-project\face-api\model_weights\face-yolov8-m.pt',
    encoder_model_path=r'C:\Users\jeffg\face-project\face-api\model_weights\feifei_face.pt',
    emotion_model_path=r"C:\Users\jeffg\face-project\face-api\model_weights\face-emotion.pth",
    
    age_proto_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\age_detector\age_deploy.prototxt', 
    age_model_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\age_detector\age_net.caffemodel', 
    gender_proto_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\gender_detector\gender_deploy.prototxt', 
    gender_model_path = r'C:\Users\jeffg\face-project\face-api\model_weights\age_gender_models\gender_detector\gender_net.caffemodel', 

    device='cuda'
)
os.chdir(now_path)

loaded finetune model , output to 8


In [4]:
fold_0_data = pd.read_csv(r"C:\Users\jeffg\face-project\dataset\AdienceGender\fold_0_data.txt", delimiter="\t")
fold_1_data = pd.read_csv(r"C:\Users\jeffg\face-project\dataset\AdienceGender\fold_1_data.txt", delimiter="\t")
fold_2_data = pd.read_csv(r"C:\Users\jeffg\face-project\dataset\AdienceGender\fold_2_data.txt", delimiter="\t")
fold_3_data = pd.read_csv(r"C:\Users\jeffg\face-project\dataset\AdienceGender\fold_3_data.txt", delimiter="\t")
fold_4_data = pd.read_csv(r"C:\Users\jeffg\face-project\dataset\AdienceGender\fold_4_data.txt", delimiter="\t")
label_data = pd.concat((
    fold_0_data, 
    fold_1_data, 
    fold_2_data,
    fold_3_data,
    fold_4_data,
)).dropna(subset=['age', "gender"])

In [5]:
gender_map = {
    'f': "female",
    'm': "male",
    'u': "unknow_gender",
}

label_data["gender"] = label_data["gender"].apply(lambda x: gender_map[x])

In [6]:
# age category classes, there are 12 age groups
ages = ["(0, 2)", "(4, 6)", "(8, 12)", "(15, 20)", "(21, 24)", "(25, 32)",
             "(33, 37)", "(38, 43)", "(44, 47)", "(48, 53)", "(54, 59)", "(60, 100)"]



# Since there are labels that do not match the classes stated, need to fix them
ages_to_fix = {'35': ages[6], '3': ages[0], '55': ages[10], '58': ages[10],
                    '22': ages[4], '13': ages[2], '45': ages[8], '36': ages[6],
                    '23': ages[4], '57': ages[10], '56': ages[10], '2': ages[0],
                    '29': ages[5], '34': ages[6], '42': ages[7], '46': ages[8],
                    '32': ages[5], '(38, 48)': ages[7], '(38, 42)': ages[7],
                    '(8, 23)': ages[2], '(27, 32)': ages[5]}

label_data["age_fixed"] = label_data["age"].apply(lambda x: ages_to_fix[x] if not x in ages else x)

In [7]:
label_data["gender"].drop_duplicates()

0              female
1                male
2110    unknow_gender
Name: gender, dtype: object

In [8]:
label_data.to_dict("records")[-1]

{'user_id': '50458575@N08',
 'original_image': '9429464468_1bfc39ecfb_o.jpg',
 'face_id': 2281,
 'age': '(25, 32)',
 'gender': 'female',
 'x': 653,
 'y': 1023,
 'dx': 555,
 'dy': 554,
 'tilt_ang': 5,
 'fiducial_yaw_angle': 30,
 'fiducial_score': 70,
 'age_fixed': '(25, 32)'}

In [9]:
# X_train, X_test, y_train, y_test = train_test_split(face_img_list, face_label_list, test_size=0.33, random_state=42)

In [10]:
train_data, test_data = train_test_split(label_data.to_dict("records"), test_size=0.33, random_state=42)

In [11]:
len(train_data)

12429

In [12]:
len(test_data)

6122

In [13]:
data_path = r"C:\Users\jeffg\face-project\dataset\AdienceGender\aligned"

In [15]:
for label_payload in tqdm(train_data):
    img = Image.open(os.path.join(data_path, label_payload["user_id"],  "landmark_aligned_face.%s.%s"%(label_payload["face_id"], label_payload["original_image"])))
    img_array = np.array(img)
    faces = face_api_manager.handle(img_array)
    if len(faces) > 0:
        face = faces[0]
        x1,y1,x2,y2  = face.xyxy
        face_img = img.crop( face.xyxy)
    
        ## 年紀
        os.makedirs(os.path.join("AdienceGender_age", "train" , label_payload["age_fixed"]) , exist_ok=True)
        face_img.save(
            os.path.join(
                "AdienceGender_age",
                "train", 
                label_payload["age_fixed"], 
                "landmark_aligned_face.%s.%s"%(label_payload["face_id"], label_payload["original_image"])
            )
        )
    
        ## Gender
        os.makedirs(os.path.join("AdienceGender_gender", "train" , label_payload["gender"]) , exist_ok=True)
        face_img.save(
            os.path.join(
                "AdienceGender_gender", 
                "train" , 
                label_payload["gender"], 
                "landmark_aligned_face.%s.%s"%(label_payload["face_id"], label_payload["original_image"])
            )
        )
        

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12429/12429 [45:13<00:00,  4.58it/s]


In [14]:
for label_payload in tqdm(test_data):
    img = Image.open(os.path.join(data_path, label_payload["user_id"],  "landmark_aligned_face.%s.%s"%(label_payload["face_id"], label_payload["original_image"])))
    img_array = np.array(img)
    faces = face_api_manager.handle(img_array)
    if len(faces) > 0:
        face = faces[0]
        x1,y1,x2,y2  = face.xyxy
        face_img = img.crop( face.xyxy)
    
        ## 年紀
        os.makedirs(os.path.join("AdienceGender_age", "test" , label_payload["age_fixed"]) , exist_ok=True)
        face_img.save(
            os.path.join(
                "AdienceGender_age",
                "test", 
                label_payload["age_fixed"], 
                "landmark_aligned_face.%s.%s"%(label_payload["face_id"], label_payload["original_image"])
            )
        )
    
        ## Gender
        os.makedirs(os.path.join("AdienceGender_gender", "test" , label_payload["gender"]) , exist_ok=True)
        face_img.save(
            os.path.join(
                "AdienceGender_gender", 
                "test" , 
                label_payload["gender"], 
                "landmark_aligned_face.%s.%s"%(label_payload["face_id"], label_payload["original_image"])
            )
        )
        

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6122/6122 [20:24<00:00,  5.00it/s]
